In [14]:
import os

In [15]:
%pwd

'd:\\sign_language_conversion'

In [16]:
os.chdir("../")

In [17]:
%pwd

'd:\\'

In [18]:
os.environ["MLFLOW_TRACKING_URI"]="MLFLOW_TRACKING_URI=https://dagshub.com/Rahulagowda004/Facial_review_system.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rahulagowda004"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d15121a9b37945a700eae5385a0ae54fa810a813"

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str

In [20]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories, save_json

In [21]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="model/model.pkl",
            training_data="artifacts/prepare_dataset/root_dir/data.pickle",
            mlflow_uri="MLFLOW_TRACKING_URI=https://dagshub.com/Rahulagowda004/Facial_review_system.mlflow",
            all_params=self.params
        )
        return eval_config

In [22]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    @staticmethod
    def load_model(path: Path) -> pickle:
        model_dict = pickle.load(open('model/model.pkl', 'rb'))
        model = model_dict['model']
        return model
    

    def evaluation(self):
        
        data_dict = pickle.load(open('artifacts/prepare_dataset/root_dir/data.pickle', 'rb'))
        data = np.asarray(data_dict['data'])
        labels = np.asarray(data_dict['labels'])
        
        x_test, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)
        
        self.model = self.load_model(self.config.path_of_model)
        self.y_pred = self.model.predict(x_test)
        self.score = accuracy_score(self.y_pred, y_test)
        self.save_score()

    def save_score(self):
        scores = {"accuracy": self.score*100}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"accuracy": self.score}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "SGC_CONVERTER", registered_model_name="RANDOM_FOREST")
            else:
                mlflow.keras.log_model(self.model, "model")

In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'